In [2]:
from model.iGPT import iGPT,iGPTConfig, NotMyModel
import torch

config = iGPTConfig(
    block_size=256,
    vocab_size=50304,
    n_layer_main=12,
    n_layer_idea=12,
    n_head=8,
    n_embd_main = 768,          
    n_embd_idea = 512,
    idea_dim=768,
)

# model = NotMyModel(config)
model = NotMyModel.load_from_checkpoint("experiments/lightning_logs/version_0/checkpoints/epoch=5-step=14149.ckpt")
# model.cuda()
model.eval()
model

/home/ubuntu/phusr/miniconda3/envs/gpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NotMyModel(
  (network): iGPT(
    (wte_i): Embedding(50304, 512)
    (wpe_i): Embedding(256, 512)
    (blocks_i): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=True)
          (c_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=2048, out_features=512, bias=True)
        )
      )
    )
    (ln_f_i): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (idea_head): Linear(in_features=512, out_features=768, bias=False)
    (wte_g): Embedding(50304, 768)
    (wpe_g): Embedding(256, 768)
    (blocks_g): ModuleList(
      (0-11): 12 x Block(
        (ln_1): Lay

In [13]:
import tiktoken
# Load the base encoding
enc = tiktoken.get_encoding("gpt2")
eos = 50257
# Define new special tokens
new_special_tokens = {
    "<|endofsent|>": eos,  # Make sure this ID does not conflict with existing tokens
}
# Create a new encoding with the added special tokens
extended_enc = tiktoken.Encoding(
    name="gpt2_extended",
    pat_str=enc._pat_str,  # Use the same pattern as the original encoding
    mergeable_ranks=enc._mergeable_ranks,  # Keep the same mergeable ranks
    special_tokens={**enc._special_tokens, **new_special_tokens},  # Extend special tokens
)

In [29]:
num_return_sequences = 5
max_length = 50
# mysent = "He is credited as a writer on Halo 3 ."
mysent = "Among the places he has lived since retirement are California and Grants Pass , Oregon ."
sent = extended_enc.encode(mysent)
sent = torch.tensor(sent, dtype=torch.long) # (8,)
sent = sent.unsqueeze(0)
sent = sent.cuda()

In [30]:
init_sent = extended_enc.encode("<|endofsent|>", allowed_special={'<|endofsent|>'})
init_sent = torch.tensor(init_sent, dtype=torch.long) # (8,)
init_sent = init_sent.unsqueeze(0).repeat(num_return_sequences, 1)
init_sent = init_sent.cuda()
init_sent

tensor([[50257],
        [50257],
        [50257],
        [50257],
        [50257]], device='cuda:0')

In [31]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

import torch.nn.functional as F

while init_sent.size(1) < max_length:
    # forward the model to get the logits
    # with torch.no_grad():
        logits = model((init_sent, sent)) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        init_sent = torch.cat((init_sent, xcol), dim=1)

In [32]:
mysent

'Among the places he has lived since retirement are California and Grants Pass , Oregon .'

In [33]:
# small models (4 layers decoder, 2 layer encoder) - 4 epochs
for i in range(num_return_sequences):
    tokens = init_sent[i, :max_length].tolist()
    decoded = extended_enc.decode(tokens)
    print(">", decoded)

> <|endofsent|>For the season in two years as a backup actress to appear between April 31 , 2015 through mid @-@ season episodes , with multiple DVDs and low @-@ ratings shows .<|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|>
> <|endofsent|>Despite the issues it became popular with a new brand including limited exclusive online provider ( EP sales ) , its retail @-@ based content and quality sales amounted to 2 @.<|endofsent|>.<|endofsent|><|endofsent|><|endofsent|>.<|endofsent|><|endofsent|>.<|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|>
> <|endofsent|>Despite the issue he started work and would be shown at various social organizations including : Atlanta University ( where Denny is mentioned as <unk> ) and Austin Cooder University .<|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|><|endofsent|

Hellaswag

In [3]:
import os
import json
import requests
import tiktoken
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F

# lightning module
from model.iGPT import NotMyModel  # import your LightningModule class
# same code for HellaSwag download + rendering as in your GPT-2 script
# -------------------------------------------------------------------------------

DATA_CACHE_DIR = os.path.join(os.path.dirname("data"), "hellaswag")
hellaswags = {
    "train": "https://raw.githubusercontent.com/rowanz/hellaswag/master/data/hellaswag_train.jsonl",
    "val": "https://raw.githubusercontent.com/rowanz/hellaswag/master/data/hellaswag_val.jsonl",
    "test": "https://raw.githubusercontent.com/rowanz/hellaswag/master/data/hellaswag_test.jsonl",
}

enc = tiktoken.get_encoding("gpt2")

def download_file(url: str, fname: str, chunk_size=1024):
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get("content-length", 0))
    with open(fname, "wb") as file, tqdm(
        desc=fname,
        total=total,
        unit="iB",
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)

def download(split):
    """Downloads HellaSwag into DATA_CACHE_DIR if not already present."""
    os.makedirs(DATA_CACHE_DIR, exist_ok=True)
    data_url = hellaswags[split]
    data_filename = os.path.join(DATA_CACHE_DIR, f"hellaswag_{split}.jsonl")
    if not os.path.exists(data_filename):
        print(f"Downloading {data_url} to {data_filename}...")
        download_file(data_url, data_filename)

def iterate_examples(split):
    download(split)
    with open(os.path.join(DATA_CACHE_DIR, f"hellaswag_{split}.jsonl"), "r") as f:
        for line in f:
            example = json.loads(line)
            yield example

def render_example(example):
    """Same as your GPT-2 code, returns (data, tokens, mask, label)."""
    ctx = example["ctx"]
    label = example["label"]
    endings = example["endings"]

    data = {
        "label": label,
        "ctx_tokens": None,
        "ending_tokens": [],
    }
    # Tokenize context
    ctx_tokens = enc.encode(ctx)
    data["ctx_tokens"] = ctx_tokens

    # Tokenize each of the 4 endings
    tok_rows = []
    mask_rows = []
    for end in endings:
        end_tokens = enc.encode(" " + end)  # note: leading space
        tok_rows.append(ctx_tokens + end_tokens)
        mask_rows.append([0]*len(ctx_tokens) + [1]*len(end_tokens))
        data["ending_tokens"].append(end_tokens)

    # Collate into a 4 x max_len shape
    max_len = max(len(row) for row in tok_rows)
    tokens = torch.zeros((4, max_len), dtype=torch.long)
    mask = torch.zeros((4, max_len), dtype=torch.long)
    for i, (tok_row, mask_row) in enumerate(zip(tok_rows, mask_rows)):
        tokens[i, :len(tok_row)] = torch.tensor(tok_row)
        mask[i, :len(mask_row)] = torch.tensor(mask_row)

    return data, tokens, mask, label

In [6]:
@torch.no_grad()
def evaluate_iGPT(model_module, device="cuda"):
    """
    Loads your trained iGPT model from a Lightning checkpoint, then
    evaluates on the HellaSwag val set. We measure two scores:

    1) 'acc' using sum of cross-entropy loss over each candidate (pred = argmin sum).
    2) 'acc_norm' using average cross-entropy over each candidate region (pred = argmin mean).
    """


    # For convenience, we'll directly call model_module.network() 
    # inside our loop below. If your model requires anything special 
    # for generation or inference, adapt as needed.

    # 2) Setup counters
    num_correct_norm = 0
    num_correct = 0
    num_total = 0

    # 3) Iterate over HellaSwag val examples
    for example in iterate_examples("val"):
        data, tokens, mask, label = render_example(example)
        tokens = tokens.to(device)  # shape: (4, max_len)
        mask = mask.to(device)      # shape: (4, max_len)

        # iGPT expects two inputs: x, ix
        # We'll feed 'tokens' as x, and supply a dummy 'ix' 
        # because we don't have any "idea tokens" at test time.
        B, T = tokens.size()  # B = 4
        ix_dummy = torch.zeros((B, 1), dtype=torch.long, device=device)

        # Forward pass to get logits: shape (B, T, vocab_size)
        logits = model_module((tokens, ix_dummy))

        # 4) Autoregressive loss
        # same procedure as the GPT-2 script
        shift_logits = logits[..., :-1, :].contiguous()   # (4, T-1, vocab_size)
        shift_tokens = tokens[..., 1:].contiguous()       # (4, T-1)
        flat_shift_logits = shift_logits.view(-1, shift_logits.size(-1))
        flat_shift_tokens = shift_tokens.view(-1)
        shift_losses = F.cross_entropy(
            flat_shift_logits, 
            flat_shift_tokens,
            reduction='none'
        )
        shift_losses = shift_losses.view(B, -1)  # shape (4, T-1)

        # apply the mask
        shift_mask = mask[..., 1:].contiguous()  # also shape (4, T-1)
        masked_shift_losses = shift_losses * shift_mask

        # sum/average over the candidate region
        sum_loss = masked_shift_losses.sum(dim=1)          # shape (4,)
        avg_loss = sum_loss / shift_mask.sum(dim=1)        # shape (4,)
        
        # the best candidate is the one with the lowest sum_loss
        pred = sum_loss.argmin().item()
        pred_norm = avg_loss.argmin().item()

        # 5) Accumulate stats
        num_total += 1
        num_correct += int(pred == label)
        num_correct_norm += int(pred_norm == label)

        # (Optional) debug prints
        # if num_total <= 10:
        #     print("---")
        #     print(f"Context:\n {example['ctx']}")
        #     print("Endings:")
        #     for i, end in enumerate(example["endings"]):
        #         print(f"{i} (loss: {avg_loss[i].item():.4f}) => {end}")
        #     print(f"Predicted: {pred_norm}, actual: {label}")

        # (Optional) show running accuracy
        if num_total % 100 == 0:
            print(f"Processed {num_total} examples. " 
                  f"acc: {num_correct/num_total:.4f}, "
                  f"acc_norm: {num_correct_norm/num_total:.4f}")

    # 6) Final results
    print(f"Done! Evaluated {num_total} examples.")
    acc = num_correct/num_total
    acc_norm = num_correct_norm/num_total
    print(f"Final Acc: {acc:.4f}  Acc_norm: {acc_norm:.4f}")

In [7]:
evaluate_iGPT(model)

Processed 100 examples. acc: 0.2500, acc_norm: 0.1800
Processed 200 examples. acc: 0.2750, acc_norm: 0.2450
Processed 300 examples. acc: 0.2867, acc_norm: 0.2667
Processed 400 examples. acc: 0.2650, acc_norm: 0.2725
Processed 500 examples. acc: 0.2640, acc_norm: 0.2680
Processed 600 examples. acc: 0.2950, acc_norm: 0.2867
Processed 700 examples. acc: 0.2871, acc_norm: 0.2800
Processed 800 examples. acc: 0.2750, acc_norm: 0.2900
Processed 900 examples. acc: 0.2756, acc_norm: 0.2967
Processed 1000 examples. acc: 0.2720, acc_norm: 0.2910
Processed 1100 examples. acc: 0.2682, acc_norm: 0.2882
Processed 1200 examples. acc: 0.2717, acc_norm: 0.2850
Processed 1300 examples. acc: 0.2746, acc_norm: 0.2815
Processed 1400 examples. acc: 0.2729, acc_norm: 0.2800
Processed 1500 examples. acc: 0.2653, acc_norm: 0.2733
Processed 1600 examples. acc: 0.2637, acc_norm: 0.2669
Processed 1700 examples. acc: 0.2718, acc_norm: 0.2641
Processed 1800 examples. acc: 0.2756, acc_norm: 0.2650
Processed 1900 exam

Save to huggingface

In [ ]:
from huggingface_hub import create_repo, upload_folder

repo_id = "phusroyal/my-lightning-checkpoints"
folder_path = "experiments/lightning_logs/version_010"  # wherever your files are

# If you haven't created the repo yet:
create_repo(repo_id, exist_ok=True)

# Upload all files in `folder_path` to the Hugging Face Hub
upload_folder(
    repo_id=repo_id,
    folder_path=folder_path,
    commit_message="Upload igptv010"
)


epoch=3-step=83166.ckpt:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

epoch=3-step=83266.ckpt:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

events.out.tfevents.1737308482.trustworthy-mbzuai.3253248.0:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
from huggingface_hub import hf_hub_download
from model.iGPT import NotMyModel  # your custom LightningModule

repo_id = "phusroyal/my-lightning-checkpoints"
# Adjust the filename if your checkpoint file has a different name
filename = "checkpoints/epoch=4-step=13604.ckpt"

# This downloads the file and returns the local path
ckpt_path = hf_hub_download(repo_id=repo_id, filename=filename)

# Now load it in your LightningModule
model_pl = NotMyModel.load_from_checkpoint(ckpt_path)

# Put model in eval mode, move to GPU/CPU, etc.
model_pl.eval()
model_pl.to("cuda")


/home/ubuntu/phusr/miniconda3/envs/gpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NotMyModel(
  (network): iGPT(
    (wte_i): Embedding(50304, 512)
    (wpe_i): Embedding(256, 512)
    (blocks_i): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=True)
          (c_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=2048, out_features=512, bias=True)
        )
      )
    )
    (ln_f_i): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (idea_head): Linear(in_features=512, out_features=768, bias=False)
    (wte_g): Embedding(50304, 768)
    (wpe_g): Embedding(256, 768)
    (blocks_g): ModuleList(
      (0-11): 12 x Block(
        (ln_1): Lay